In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [2]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [3]:
df_fake.head()


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head(5)


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
df_fake["class"] = 0
df_true["class"] = 1

In [6]:
df_fake.shape

(23481, 5)

In [7]:
df_merge = pd.concat([df_fake, df_true], axis=0)

In [8]:
df = df_merge.drop(["title", "subject", "date"], axis=1)

In [9]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [10]:
df = df.sample(frac=1).reset_index(drop=True)

In [11]:
df.head(10)

,text,class
0,WASHINGTON (Reuters) - FBI Director James Come...,1
1,WASHINGTON (Reuters) - United Nations envoy Je...,1
2,"While Obama golfs in Martha s Vineyard, yet an...",0
3,WASHINGTON (Reuters) - Democratic Senator Al F...,1
4,WASHINGTON (Reuters) - U.S. Chief Justice John...,1
5,As President Obama is threatening and acting o...,0
6,"Earlier on Tuesday, Donald Trump decided to ac...",0
7,"If you ve spent much time on social media, you...",0
8,NEW YORK (Reuters) - Former President Bill Cli...,1
9,Israeli Prime Minister Benjamin Netanyahu has ...,0


In [12]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [item for item in text if not isinstance(item, int)]
    return text

In [13]:
df.head(10)

,text,class
0,WASHINGTON (Reuters) - FBI Director James Come...,1
1,WASHINGTON (Reuters) - United Nations envoy Je...,1
2,"While Obama golfs in Martha s Vineyard, yet an...",0
3,WASHINGTON (Reuters) - Democratic Senator Al F...,1
4,WASHINGTON (Reuters) - U.S. Chief Justice John...,1
5,As President Obama is threatening and acting o...,0
6,"Earlier on Tuesday, Donald Trump decided to ac...",0
7,"If you ve spent much time on social media, you...",0
8,NEW YORK (Reuters) - Former President Bill Cli...,1
9,Israeli Prime Minister Benjamin Netanyahu has ...,0


In [14]:
df["text"] = df["text"].apply(wordopt)

In [15]:
x = df["text"]
y = df["class"]

In [16]:
#Splitting Training and Testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [22]:
#convert to string
x_train = x_train.astype(str)
y_train = y_train.astype(str)
x_test = x_test.astype(str)
y_test = y_test.astype(str)


In [23]:
#Conver to vector
from sklearn.feature_extraction.text import CountVectorizer
vectrorization = CountVectorizer(lowercase=False)
xv_train = vectrorization.fit_transform(x_train)
yv_train = vectrorization.fit_transform(y_train)

ValueError: empty vocabulary; perhaps the documents only contain stop words